# Demonstration View

View a demonstration by sliding through the frames.
This also plots the z height.

In [ ]:
list(state_recording.keys())
gripper_states = state_recording["gripper_states"]
print(gripper_states.astype(int))
diff = np.diff(gripper_states)
gripper_state_changes = np.where(diff)[0]+1

switch_frame = gripper_state_changes[0]

print(gripper_states[switch_frame-1], gripper_states[switch_frame])
print(switch_frame)

# Save images to Files

# Mask View

Mask out the foreground object so that foreground specific flow can be calculated.

In [ ]:
import glob

%matplotlib notebook
from ipywidgets import *
import matplotlib.pyplot as plt
import numpy as np

run_files = glob.glob("eval/*.npz")

runs = []

for fn in sorted(run_files):
    runs.append(np.load(fn))

rewards = [r["reward"][-1] for r in runs]
sr = np.mean(rewards)
print("Rewards: ", rewards)
print("Success rate: ", sr)

    
num_episodes = len(runs)


fig, ax = plt.subplots(1,1)
line = ax.imshow(runs[0]["state"][0])
ax.set_axis_off()


def update(w,i):
    frames = len(runs[i]["state"])
    w_new = min(w,frames-1)
    image = runs[i]["state"][w_new].copy()
    line.set_data(image)
    fig.canvas.draw_idle()
    print(w, i)
    
slider_w = widgets.IntSlider(min=0,max=600-1,step=10,value=0,
                             layout=Layout(width='70%'))

slider_i = widgets.IntSlider(min=0,max=num_episodes-1,step=1,value=0,
                             layout=Layout(width='70%'))

interact(update,w=slider_w, i=slider_i)


In [ ]:
threshold = 0.6
masks = np.zeros(image_recording.shape[:3],dtype=bool)

if recording == "stack_recordings/episode_118":
    switch_frame = 65
elif recording == "stack_recordings/episode_129":
    switch_frame = 65
elif recording == "stack_recordings/episode_156":
    switch_frame = 80
else:
    switch_frame = 1e6
    
for i in range(len(image_recording)):
    if i < switch_frame:
        color_index = 2
    else:
        color_index = 0
    mask = get_mask(image_recording[i], threshold, color_index)
    masks[i] = mask

print(np.mean(masks) * 100, "percent of pixels fg")
mask_fn = image_recording_fn.replace("_img.npz","_mask.npz")
np.savez(mask_fn, mask=masks)
print("Saved to",mask_fn)

In [ ]:
fig, ax = plt.subplots(1,1)
line = ax.imshow(masks[25])
ax.set_axis_off()

def update(i):
    image = image_recording[i].copy()
    mask = masks[i]
    image[np.logical_not(mask)] = 255,255,255
    line.set_data(image)
    fig.canvas.draw_idle()
    
slider_i2 = widgets.IntSlider(min=0,max=num_frames,step=1,value=25,
                             layout=Layout(width='70%'))

interact(update, i=slider_i2)

In [ ]:
# This should select a bounding box.

index_A = 25
index_B = 5

fig, ((ax, ax2), (ax3, ax4)) = plt.subplots(2, 2)
ax.imshow(image_recording[index_A])
ax.set_axis_off()
tmp = image_recording[index_A].copy()
tmp[20:60,35:55] = 255

ax2.imshow(tmp)
ax2.set_axis_off()

ax3.imshow(image_recording[index_B])

